In [40]:
import base64
import secrets
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [41]:
def parse_to_dict(data):
    data_splits = [data_block.split(b"=") for data_block in data.split(b"&")]
    encoded_data = {data_split[0]:data_split[1] for data_split in data_splits}
    return encoded_data

In [42]:
test_data = b"foo=bar&baz=qux&zap=zazzle"
print("The encoded data is", parse_to_dict(test_data))

The encoded data is {b'foo': b'bar', b'baz': b'qux', b'zap': b'zazzle'}


In [43]:
BLOCK_LENGTH = 16

In [44]:
def generate_random_bytes(bytes_length):
    random_bytes = secrets.token_bytes(bytes_length)
    return random_bytes

def reject_input(data):
    if b"&" in data or b"=" in data:
        return False
    else:
        return True
    
def encode_profile(email):
    assert reject_input(email) 
    parsed_profile = b"email=" + email + b"&uid=10&role=user"
    return parsed_profile
    
def profile_for(email):    
    parsed_profile = encode_profile(email)
    return parse_to_dict(parsed_profile)

In [45]:
KEY = generate_random_bytes(BLOCK_LENGTH)

def encrypt_ECB(email):
    message= encode_profile(email) 
    cipher = AES.new(KEY, AES.MODE_ECB)
    return cipher.encrypt(pad(message, BLOCK_LENGTH))

def parse_from_dict(message):
    res = []
    for key, value in message:
        res.append("key"+b"="+"value")
        return "&".join(res)

def decrypt(encrypted_message):
    cipher = AES.new(KEY, AES.MODE_ECB)
    decrypted_message = unpad(cipher.decrypt(encrypted_message), BLOCK_LENGTH)
    return decrypted_message

In [46]:
def find_repeat_start_location():
    message = encrypt_ECB(b"A"*(BLOCK_LENGTH*3))
    for i in range(len(message) - 2*BLOCK_LENGTH):
        if message[i:i+BLOCK_LENGTH] == message[i+BLOCK_LENGTH:i+2*BLOCK_LENGTH]:
            return i

def find_input_location():
    loc = find_repeat_start_location()
    start_length = BLOCK_LENGTH * 3
    while start_length > 0:
        message = encrypt_ECB(b"A"*(start_length))
        if message[loc:loc+BLOCK_LENGTH] == message[loc+BLOCK_LENGTH:loc+2*BLOCK_LENGTH]:
            start_length -= 1
        else:
            break
    return start_length

In [47]:
location_idx = find_input_location()
n_As = location_idx + 1 - BLOCK_LENGTH
MESSAGE_PIECE1 = encrypt_ECB(b"max@gmail.")[:BLOCK_LENGTH]                      # product "email=max@gmail."
MESSAGE_PIECE2 = encrypt_ECB(b"A"*(n_As) + b"com")[2*BLOCK_LENGTH:3*BLOCK_LENGTH] # produce "com&uid=10&role="
MESSAGE_PIECE3 = encrypt_ECB(b"A"*n_As + b"admin" + b"\x0b"*11)[2*BLOCK_LENGTH:3*BLOCK_LENGTH] # produce "admin\x0b\x0b\x0b\x0b\x0b\x0b"

In [48]:
FORGED_MESSAGE = MESSAGE_PIECE1 + MESSAGE_PIECE2 + MESSAGE_PIECE3
decrypt(FORGED_MESSAGE)

b'email=max@gmail.com&uid=10&role=admin'